# Problem 3

### Data preprocessing

In [1]:
#combine all input files
import glob
read_files = glob.glob("C:\\Users\\81468\\OneDrive\\Documents\\CS6120\\homework\\a1\\train\\*.txt")
with open("result.txt", "wb") as outfile:
    for f in read_files:
        with open(f, "rb") as infile:
            outfile.write(infile.read())

In [2]:
#remove space lines
with open('result.txt','r+') as file: 
    for line in file:
        if not line.isspace():
            file.write(line)

In [3]:
#read file
f_open = open(r"result.txt","r")
data = f_open.read()
f_open.close()

In [4]:
#convert punctuations to spaces
import re, string
re_p = re.compile('[%s]' % re.escape(string.punctuation))
data = re_p.sub(' ', data) 

In [5]:
#replace duplicate spaces and \n to single space
data = re.sub(r'[\s]',' ',data) 
data = re.sub(r' {2,}',' ',data) 

In [6]:
def count(s):
    c=dict.fromkeys(set(s),0)
    for i in range(0,len(s)):
        c[s[i]] = c[s[i]]+1
    return c

In [7]:
counts = count(data)

In [8]:
#replace character that appears <=5 as "*"
for char in set(data):
    if counts[char] <= 5:
        data = data.replace(char,'*')

### 3.1 [10 points]
#### Across all ﬁles in the directory (counted together), report the unigram, bigram, and trigram character counts. You can submit them in separate ﬁles.


In [9]:
#function to generate ngrams
def ngram_generate(s,n):
    ngram = []
    for i in range(len(s)-(n-1)):
        ngram.append(s[i:i+n]) 
    return ngram

In [10]:
#generate bigram and trigram
bigram = ngram_generate(data,2)
trigram = ngram_generate(data,3)

In [11]:
unigram_counts = count(data)

In [12]:
with open('unigram_counts.txt', 'w') as f:
    for key, value in unigram_counts.items():
        f.write('%s:%s\n' % (key, value))

In [13]:
bigram_counts = count(bigram)

In [14]:
with open('bigram_counts.txt', 'w') as f:
    for key, value in bigram_counts.items():
        f.write('%s:%s\n' % (key, value))

In [15]:
trigram_counts = count(trigram)

In [16]:
with open('trigram_counts.txt', 'w') as f:
    for key, value in trigram_counts.items():
        f.write('%s:%s\n' % (key, value))

### 3.2 [10 points]
#### Calculate the perplexity for each ﬁle in the test set using linear interpolation smoothing method. 

##### Preprocessing tr_set and hd_set

In [17]:
#split training data by sentence
import random
with open("result.txt", "r") as f:
    data_ttl = f.read().split('.')
random.seed(1)
random.shuffle(data_ttl)
tr_set = data_ttl[:int(len(data_ttl)*.80)]
hd_set = data_ttl[int(len(data_ttl)*.80)+1:]

In [18]:
#convert punctuations to spaces
tr_set = re_p.sub(' ', '.'.join(tr_set)) 
hd_set = re_p.sub(' ', '.'.join(hd_set))
#replace duplicate spaces and \n to single space
tr_set = re.sub(r'[\s]',' ',tr_set) 
tr_set = re.sub(r' {2,}',' ',tr_set) 
hd_set = re.sub(r'[\s]',' ',hd_set) 
hd_set = re.sub(r' {2,}',' ',hd_set) 

In [19]:
counts_new = count(tr_set)

In [20]:
#replace character that appears <=5 as "*"
for char in set(tr_set):
    if counts_new[char] <= 5:
        tr_set = tr_set.replace(char,'*')
for char in set(hd_set):
    if counts_new[char] <= 5 or char not in counts_new:
        hd_set = hd_set.replace(char,'*')

##### Calculate probability on training set

In [21]:
def unigram_prob(ngram_counts):
    prob_ngram = dict.fromkeys(ngram_counts)
    for key in ngram_counts.keys():
        prob_ngram[key] = ngram_counts[key]/sum(ngram_counts.values())
    return prob_ngram

In [22]:
def ngram_prob(a,b,n):
    prob_ngram = dict.fromkeys(a)
    for key in a:
        prob_ngram[key] = a[key]/b[key[0:n-1]]
    return prob_ngram

In [23]:
#Generate ngrams and count ngrams on training set
tr_unigram_counts = count(tr_set)
tr_bigram = ngram_generate(tr_set,2)
tr_bigram_counts = count(tr_bigram)
tr_trigram = ngram_generate(tr_set,3)
tr_trigram_counts = count(tr_trigram)

In [24]:
#Calculate the probability
tr_prob_unigram = unigram_prob(tr_unigram_counts)
tr_prob_bigram = ngram_prob(tr_bigram_counts,tr_unigram_counts,2)
tr_prob_trigram = ngram_prob(tr_trigram_counts,tr_bigram_counts,3)

##### Create lambda combinations

In [25]:
#lambdas
l = []
for i in range(1,9):
    for j in range(1,10-i):
        l.append([i/10, j/10, 1-i/10-j/10])

##### Calculate prob on held out set

In [26]:
hd_trigram = ngram_generate(hd_set,3)

In [27]:
#calculate the interpolation estimated probability for each trigram
#format of pro_hat: [{tri1:p1_1,tri2:p2_1,...},{tri1:p1_2,tri2:p2_2,...},...]
pro_hat = []
for i in range(len(l)):
    pro_hat.append(dict.fromkeys(hd_trigram))
    for key in hd_trigram:
        if key in tr_prob_trigram.keys():
            pro_hat[i][key] = l[i][0]*tr_prob_trigram[key] + l[i][1]*tr_prob_bigram[key[-2:]] + l[i][2]*tr_prob_unigram[key[-1]]
        elif key[-2:] in tr_prob_bigram.keys():
            pro_hat[i][key] = l[i][1]*tr_prob_bigram[key[-2:]] + l[i][2]*tr_prob_unigram[key[-1]]
        else:
            pro_hat[i][key] = l[i][2]*tr_prob_unigram[key[-1]]

In [28]:
#calculate log(p(hd_set))
import math
log_p_hd = []
for j in range(len(l)):
    p=0
    for i in range(len(hd_set)-3):
        p = p + math.log(pro_hat[j][hd_set[i:i+3]])
    log_p_hd.append(p)

In [29]:
#best lambda combination
l[log_p_hd.index(max(log_p_hd))] 

[0.8, 0.1, 0.09999999999999995]

The best lambda combination is 0.8 for trigram, 0.1 for bigram and 0.1 for unigram, as it gives the highest probability on held out set.

##### Calculate probability on total training data

In [30]:
prob_unigram = unigram_prob(unigram_counts)
prob_bigram = ngram_prob(bigram_counts,unigram_counts,2)
prob_trigram = ngram_prob(trigram_counts,bigram_counts,3)

##### Processing test data

In [31]:
import glob
read_files = glob.glob("C:\\Users\\81468\\OneDrive\\Documents\\CS6120\\homework\\a1\\test_data\\*")
testdata = []
for i in range(len(read_files)):
    with open(read_files[i], "rb") as infile:
        for line in infile:
            if not line.isspace():
                testdata.append(infile.read().decode('utf-8'))

In [32]:
#convert punctuations to spaces
#replace duplicate spaces and \n to single space
#replace character that appears <=5 as "*"
for i in range(len(testdata)):
    testdata[i] = re_p.sub(' ', testdata[i]) 
    testdata[i] = re.sub(r'[\s]',' ',testdata[i]) 
    testdata[i] = re.sub(r' {2,}',' ',testdata[i]) 
    for char in set(testdata[i]):
        if (char in counts and counts[char] <= 5) or char not in counts:
            testdata[i] = testdata[i].replace(char,'*')

In [33]:
#for each test file, calculate the estimated trigram prob and perplexity 
test_trigram = []
test_pp = []
test_pro_hat=[]
for i in range(len(testdata)):
    test_trigram.append(ngram_generate(testdata[i],3))
    test_pro_hat.append(dict.fromkeys(test_trigram[i]))
    for key in test_trigram[i]:
        if key in prob_trigram.keys():
            test_pro_hat[i][key] = 0.8*prob_trigram[key] + 0.1*prob_bigram[key[-2:]] + 0.1*prob_unigram[key[-1]]
        elif key[-2:] in prob_bigram.keys():
            test_pro_hat[i][key] = 0.1*prob_bigram[key[-2:]] + 0.1*prob_unigram[key[-1]]
        else:
            test_pro_hat[i][key] = 0.1*prob_unigram[key[-1]]        
    pp = 0
    for j in range(len(testdata[i])-3):
        pp = pp + math.log(1/test_pro_hat[i][testdata[i][j:j+3]])                       
    perp = math.exp(pp/(len(testdata[i])-3))
    test_pp.append(perp)

In [34]:
perp_report = {}
for i in range(len(test_pp)):
    perp_report[read_files[i][-4:]] = test_pp[i]

In [35]:
with open('perp_report_intp.txt', 'w') as f:
    for key, value in perp_report.items():
        f.write('%s, %s \n' % (key,value))

In [36]:
from heapq import nlargest
largest = nlargest(50, test_pp)
largest_pfile = {}
for i in range(0,50):
    largest_pfile[read_files[test_pp.index(largest[i])][-4:]] = largest[i] 

In [37]:
with open('largest_pfile_intp.txt', 'w') as f:
    for key, value in largest_pfile.items():
        f.write('%s, %s \n' % (key,value))

After check, they are all French files.

### 3.3 [10 points]
#### Build another language model with add-λ smoothing (use λ = 0.1).

In [38]:
#Add 0.1 smoothing
tri={}
V = len(unigram_counts)
uniq = list(set(unigram_counts.keys()))
for i in range(len(unigram_counts)):
    for j in range(len(unigram_counts)):
        for k in range(len(unigram_counts)):
            k3 = uniq[i]+uniq[j]+uniq[k]
            k2 = uniq[i]+uniq[j]
            if k3 in trigram_counts:
                tri[k3] = (trigram_counts[k3]+0.1)/(bigram_counts[k2]+0.1*V)
            elif k2 in bigram_counts:
                tri[k3] = 0.1/(bigram_counts[k2]+0.1*V)
            else:
                tri[k3] = 1/V

In [39]:
#for each test file, calculate the perplexity 
import math
test_pp_add1 = []
for i in range(len(testdata)):
    p = 0
    for j in range(len(testdata[i])-3):
        p = p + math.log(1/tri[testdata[i][j:j+3]])                           
    perp = math.exp(p/(len(testdata[i])-3))
    test_pp_add1.append(perp)

In [40]:
perp_report_add1 = {}
for i in range(len(test_pp_add1)):
    perp_report_add1[read_files[i][-4:]] = test_pp_add1[i]

In [41]:
with open('perp_report_addk.txt', 'w') as f:
    for key, value in perp_report_add1.items():
        f.write('%s, %s \n' % (key,value))

In [42]:
from heapq import nlargest
largest_add1 = nlargest(50, test_pp_add1)
largest_pfile_add1 = {}
for i in range(0,50):
    largest_pfile_add1[read_files[test_pp_add1.index(largest_add1[i])][-4:]] = largest_add1[i] 

In [43]:
with open('largest_pfile_addk.txt', 'w') as f:
    for key, value in largest_pfile_add1.items():
        f.write('%s, %s \n' % (key,value))

After check, they are all French files.